# 快速入门 GPT-4 Vison

从历史上看，语言模型系统仅接受**文本**作为输入。但是单一的输入形式，限制了大模型的应用落地范围。

随着技术发展，OpenAI 开发的 GPT-4 Turbo with Vision（简称 GPT-4V）允许模型接收**图像**作为输入，并回答关于它们的问题。

📢注意，目前在 Assistants API 中使用 GPT-4 时还不支持图像输入。

## 使用 GPT-4V 识别线上图像（URL）

![image_sample](https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg)

In [1]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
  model="gpt-4-turbo",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "介绍下这幅图?"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0])

Choice(finish_reason='length', index=0, logprobs=None, message=ChatCompletionMessage(content='这幅图呈现了一个宁静美丽的自然景观。图中的主要特点是一条木制的栈道，它穿越在郁郁葱葱的草地上，引向远处看不到的终点。草地上覆盖着茂密的绿色植被，显示出生机勃勃的春夏季节。背景中天空呈现出澄清的蓝色，点缀着几朵轻盈的白云，增添了一种宁静和平和的氛围。\n\n整个场景光线充足，阳光和蓝天与绿色植被的对比令人感觉清新愉快。这样的环境可能是理想的徒步旅行地点，提供了亲近自然和放松心情的完美机会。整体上，这幅图传达了大自然的宁静与和谐，是观赏和体验自然美景的一种', role='assistant', function_call=None, tool_calls=None))


In [2]:
response.choices[0].message.content

'这幅图呈现了一个宁静美丽的自然景观。图中的主要特点是一条木制的栈道，它穿越在郁郁葱葱的草地上，引向远处看不到的终点。草地上覆盖着茂密的绿色植被，显示出生机勃勃的春夏季节。背景中天空呈现出澄清的蓝色，点缀着几朵轻盈的白云，增添了一种宁静和平和的氛围。\n\n整个场景光线充足，阳光和蓝天与绿色植被的对比令人感觉清新愉快。这样的环境可能是理想的徒步旅行地点，提供了亲近自然和放松心情的完美机会。整体上，这幅图传达了大自然的宁静与和谐，是观赏和体验自然美景的一种'

### 封装成一个函数 query_image_description

In [3]:
def query_image_description(url, prompt="介绍下这幅图?"):
    client = OpenAI(
    api_key= os.environ.get('OPENAI_API_KEY'),
    base_url='https://api.xiaoai.plus/v1'
    )  # 初始化 OpenAI 客户端
    
    # 发送请求给 OpenAI 的聊天模型
    response = client.chat.completions.create(
        model="gpt-4-turbo",  # 指定使用的模型
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": url}},
                ],
            }
        ],
        max_tokens=300,
    )
    
    # 返回模型的响应
    return response.choices[0].message.content


### 调用函数测试

![meme_0](https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg)

In [4]:
image_url = "https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg"
content = query_image_description(image_url)
print(content)

这幅图是一种幽默风格的搞笑比较。图中展示了两种不同状态的狗。左边是一只被幻想成拥有非常发达肌肉、类似人类健美运动员身体的柴犬，配文“16岁的我，工作后的我”，表达了年轻时充满活力和梦想的状态。右边的柴犬则显得普通、略显憔悴，配有文字“好累好困、好想摸鱼、看到鸡腿就开心，不属于我、我也有小肚腩、肉肉的很可爱”，描绘了工作后可能感到疲惫、减少锻炼而略显发福的现实状态。

这种对比通常用来幽默地表达人们对于理想与现实之间差距的感慨，展示了许多人对于年轻时的憧


### 使用 GPT-4V 识别本地图像文件（Base64编码）


In [38]:
import os
from openai import OpenAI
import base64
import requests
import json

client = OpenAI(
    api_key= os.environ.get('OPENAI_API_KEY'),
    base_url='https://api.xiaoai.plus/v1'
)  # 初始化 OpenAI 客户端

def query_base64_image_description(image_path, prompt="识别下图里的内容？将图片内的文字识别并生成出来", max_tokens=1000):

    # 实现 Base64 编码
    def encode_image(path):
        with open(path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # 获取图像的 Base64 编码字符串
    base64_image = encode_image(image_path)

    # 构造请求的 HTTP Header
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {client.api_key}"
    }

    # 构造请求的负载
    payload = {
        "model": "gpt-4-turbo",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": max_tokens
    }

    # 发送 HTTP 请求
    response = requests.post("https://api.xiaoai.plus/v1/chat/completions", headers=headers, json=payload)

    # 检查响应并提取所需的 content 字段
    if response.status_code == 200:
        response_data = response.json()
        content = response_data['choices'][0]['message']['content']
        return content
    else:
        return f"Error: {response.status_code}, {response.text}"

In [40]:
content = query_base64_image_description("./images/gpt-4v.jpg")
print(content)

识别的文字内容如下：

```
坚持人民主体，
坚持和完善人民当家作主的制度。

八项要求：
要始终把人民安居乐业、安有冷暖放在心上；
要围绕和拓展产业，促进经济发展成果；
要做好高职毕业生、农民工退役军人等重点群体的就业工作；
要重视聆听群众所提建议和合理诉求；
要保持十强学生态，及相关建设的道路定力；
要树立正确的权力观、权力观、事业观。 
要鼓励各级干部，主动服务人民群体的需求，
共同落实，切实落实。
要坚持不懈对照检视，坚持不懈对照和强化落实
评价，扎实行动。
```


#### 使用 GPT-4V 识别手写体笔记

![](./images/gpt-4v.jpg)

#### 在 Jupyter 标准输出中渲染 Markdown 格式内容

In [41]:
from IPython.display import display, Markdown

# 使用 display 和 Markdown 函数显示 Markdown 内容
display(Markdown(content))

识别的文字内容如下：

```
坚持人民主体，
坚持和完善人民当家作主的制度。

八项要求：
要始终把人民安居乐业、安有冷暖放在心上；
要围绕和拓展产业，促进经济发展成果；
要做好高职毕业生、农民工退役军人等重点群体的就业工作；
要重视聆听群众所提建议和合理诉求；
要保持十强学生态，及相关建设的道路定力；
要树立正确的权力观、权力观、事业观。 
要鼓励各级干部，主动服务人民群体的需求，
共同落实，切实落实。
要坚持不懈对照检视，坚持不懈对照和强化落实
评价，扎实行动。
```

In [42]:
content = query_base64_image_description("./images/gpt-4v.jpg")
display(Markdown(content))

识别到的文字如下：

```
坚持人民民主，
坚持和完善人民代表大会制度。

八个要求：
要始终把人民安居乐业、安有冷暖放在心上；
要围绕和拓展产业、促进生产发展成果；
要做好高校毕业生、农民工等重点群体的就业工作，
要积极推动重大疫情防控政策的落实和与共体结合，
要保持和强化生态环境、并明确建设的路径；
要树立正确的权力观、政绩观、事业观；
要鼓励各级干部，特别是年轻干部主动担当作为，
共同落实、合心合力。
要坚持不懈对反腐败、坚持不懈反对和克服形式主义。
```

## Homework: 


### #1

使用 GPT-4V 识别带有手写体文字的本地图像文件，分享结果。

### #2

整合 `query_base64_image_description` 函数和 Markdown 格式渲染方法，使得输出结果更易阅读。